<a id='top'></a>
<h1 style="text-align:center;font-size:200%;;">Understanding BERT for Disaster NLP</h1>
<img src="https://cx3digital.com/wp-content/uploads/2019/11/google-bert.jpg" width="700" height="700" align="center"/>

# Introduction:<br>
### Welcome! to this kernel.In this kernel we are going to explore Google's BERT.I will try to cover all the topics which helps to understand BERT clearly.Finally,we will see how to implement BERT for disaster NLP 😊 .<br>
###  <font color='blue'>Without wasting time lets get started.</font> <br>


## Table of Contents:<br>
1. Intution behind RNN based Sequence-to-Sequence Model<br>
   1.1 Limitations of RNN'S <br>
2. Introduction to the Transformers <br>
   2.1 Transformer’s Model Architecture<br>
   2.2 Understanding self attention<br>
   2.3 Limitations of the Transformer<br>
3. Understanding BERT <br>
   3.1 How Does BERT Work?<br>
   3.2 Text processing for BERT<br>
   3.3 How to use BERT (Fine-tuning)<br>
   3.4 Takeaways<br>
   3.5 Compute considerations (training and applying)<br>
4. Implementation of BERT using TFHub<br>
   4.1 Importing necessary modules<br>
   4.2 Helper Functions<br>
   4.3 Loading BERT from the Tensorflow Hub<br>
   4.4 Loading data<br>
   4.5 Loading tokenizer from the bert layer<br>
   4.6 Encoding the text into tokens, masks, and segment flags<br>
   4.7 Model: Build, Train, Predict, Submit<br>
   
   

### References and credits:
1. [BERT Explained: State of the art language model for NLP](https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270)<br>
2. [How do Transformers Work in NLP? A Guide to the Latest State-of-the-Art Models](https://www.analyticsvidhya.com/blog/2019/06/understanding-transformers-nlp-state-of-the-art-models/)<br>
3. [Demystifying BERT: A Comprehensive Guide to the Groundbreaking NLP Framework](https://www.analyticsvidhya.com/blog/2019/09/demystifying-bert-groundbreaking-nlp-framework/)
4. [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)<br>
5. [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)<br>
6. [Disaster NLP: Keras BERT using TFHub](https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub)<br>
Thanks to all the above blogs and article contributors.Thanks you very much for you open source contibutions.

# 1. Intution behind RNN based Sequence-to-Sequence Model:
Sequence-to-sequence (seq2seq) models in NLP are used to convert sequences of Type A to sequences of Type B. For example, translation of English sentences to German sentences is a sequence-to-sequence task.<br>
Let’s take a simple example of a sequence-to-sequence model. Check out the below illustration:<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/seq2seq.gif)<br>
The above seq2seq model is converting a German phrase to its English counterpart. Let’s break it down:<br>

* Both Encoder and Decoder are RNNs
* At every time step in the Encoder, the RNN takes a word vector (xi) from the input sequence and a hidden state (Hi) from the previous time step
* The hidden state is updated at each time step
* The hidden state from the last unit is known as the context vector. This contains information about the input sequence
* This context vector is then passed to the decoder and it is then used to generate the target sequence (English phrase)
* If we use the Attention mechanism, then the weighted sum of the hidden states are passed as the context vector to the decoder

# 1.1 Limitations of RNN'S:
* Dealing with long-range dependencies is still challenging
* The sequential nature of the model architecture prevents parallelization.
* Vanishing Gradient Problem
* Issue of increasing gradients at each step called as exploding gradients.

# 2. Introduction to the Transformer:<br>
The Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution. The idea behind Transformer is to handle the dependencies between input and output with attention and recurrence completely.<br>
## 2.1 Transformer’s Model Architecture:<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-19-53-10.png) <br>
Let’s first focus on the Encoder and Decoder parts only.Now focus on the below image. The Encoder block has 1 layer of a Multi-Head Attention followed by another layer of Feed Forward Neural Network. The decoder, on the other hand, has an extra Masked Multi-Head Attention.<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-20-01-32.png)<br>


**Encoder**: The encoder is composed of a stack of N = 6 identical layers. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, positionwise fully connected feed-forward network. We employ a residual connection around each ofthe two sub-layers, followed by layer normalization. That is, the output of each sub-layer is LayerNorm(x + Sublayer(x)),where Sublayer(x) is the function implemented by the sub-layeritself. To facilitate these residual connections, all sub-layers in the model, as well as the embeddinglayers, produce outputs of dimension dmodel = 512.<br>


**Decoder**: The decoder is also composed of a stack of N = 6 identical layers. In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions less than i.<br>

The encoder and decoder blocks are actually multiple identical encoders and decoders stacked on top of each other. Both the encoder stack and the decoder stack have the same number of units.The number of encoder and decoder units is a hyperparameter <br>

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-20-03-14.png)<br>

Let’s see how this setup of the encoder and the decoder stack works:<br>

* The word embeddings of the input sequence are passed to the first encoder <br>
* These are then transformed and propagated to the next encoder<br>
* The output from the last encoder in the encoder-stack is passed to all the decoders in the decoder-stack as shown in the figure below:<br>

An important thing to note here – in addition to the self-attention and feed-forward layers, the decoders also have one more layer of Encoder-Decoder Attention layer. This helps the decoder focus on the appropriate parts of the input sequence.<br>

## 2.2 Understanding self attention: <br>
Self-attention, sometimes called intra-attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence.<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-22-31-11.png)<br>
Take a look at the above image. Can you figure out what the term “it” in this sentence refers to?<br>

Is it referring to the street or to the animal? It’s a simple question for us but not for an algorithm. When the model is processing the word “it”, self-attention tries to associate “it” with “animal” in the same sentence.<br>

Self-attention allows the model to look at the other words in the input sequence to get a better understanding of a certain word in the sequence.<br>

Self-attention is computed not once but multiple times in the Transformer’s architecture, in parallel and independently. It is therefore referred to as Multi-head Attention. The outputs are concatenated and linearly transformed as shown in the figure below:
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-22-47-53.png)<br>

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions.<br>

## 2.3 Limitations of the Transformer:<br>
Attention can only deal with fixed-length text strings. The text has to be split into a certain number of segments or chunks before being fed into the system as input
This chunking of text causes context fragmentation. For example, if a sentence is split from the middle, then a significant amount of context is lost. In other words, the text is split without respecting the sentence or any other semantic boundary <br>

But,undoutedly transformer inspired BERT and all the following breakthroughs in NLP.

# 3. Understanding BERT (Bidirectional Encoder Representations from Transformers): <br>
BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications.<br>

BERT is a **deeply bidirectional** model. Bidirectional means that BERT learns information from both the left and the right side of a token’s context during the training phase.<br>

The bidirectionality of a model is important for truly understanding the meaning of a language. Let’s see an example to illustrate this. There are two sentences in this example and both of them involve the word “bank”:<br>

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/09/sent_context.png)<br>

If we try to predict the nature of the word “bank” by only taking either the left or the right context, then we will be making an error in at least one of the two given examples.One way to deal with this is to consider both the left and the right context before making a prediction. That’s exactly what BERT does. <br>

## 3.1 How Does BERT Work? <br>
### 3.1.1 BERT’s Architecture: <br>

The BERT architecture builds on top of Transformer. We currently have two variants available:<br>

* BERT Base: 12 layers (transformer blocks), 12 attention heads, and 110 million parameters
* BERT Large: 24 layers (transformer blocks), 16 attention heads and, 340 million parameters

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/09/bert_encoder.png/)<br>

All of these Transformer layers are Encoder-only blocks.<br>

Now that we know the overall architecture of BERT, let’s see what kind of text processing steps are required before we get to the model building phase.<br>

## 3.2 Text processing for BERT:<br>
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/09/bert_emnedding.png)<br>
For bert,every input embedding is a combination of 3 embeddings:<br>

**Position Embeddings**: BERT learns and uses positional embeddings to express the position of words in a sentence. These are added to overcome the limitation of Transformer which, unlike an RNN, is not able to capture “sequence” or “order” information<br>
**Segment Embeddings**: BERT can also take sentence pairs as inputs for tasks (Question-Answering). That’s why it learns a unique embedding for the first and the second sentences to help the model distinguish between them. In the above example, all the tokens marked as EA belong to sentence A (and similarly for EB)<br>
**Token Embeddings**: These are the embeddings learned for the specific token from the WordPiece token vocabulary<br>

For a given token, its input representation is constructed by summing the corresponding token, segment, and position embeddings.Such a comprehensive embedding scheme contains a lot of useful information for the model.These combinations of preprocessing steps make BERT so versatile.<br>

When training language models, there is a challenge of defining a prediction goal. Many models predict the next word in a sequence, a directional approach which inherently limits context learning. To overcome this challenge, BERT uses two training strategies:<br>

### 1. Masked LM (MLM): <br>
Before feeding word sequences into BERT, 15% of the words in each sequence are replaced with a [MASK] token. The model then attempts to predict the original value of the masked words, based on the context provided by the other, non-masked, words in the sequence. In technical terms, the prediction of the output words requires:
1. Adding a classification layer on top of the encoder output.<br>
2. Multiplying the output vectors by the embedding matrix, transforming them into the vocabulary dimension.<br>
3. Calculating the probability of each word in the vocabulary with softmax.<br>

![](https://miro.medium.com/max/986/0*ViwaI3Vvbnd-CJSQ.png)<br>
The BERT loss function takes into consideration only the prediction of the masked values and ignores the prediction of the non-masked words. As a consequence, the model converges slower than directional models, a characteristic which is offset by its increased context awareness.<br>

### 2. Next Sentence Prediction (NSP): <br>
In the BERT training process, the model receives pairs of sentences as input and learns to predict if the second sentence in the pair is the subsequent sentence in the original document. During training, 50% of the inputs are a pair in which the second sentence is the subsequent sentence in the original document, while in the other 50% a random sentence from the corpus is chosen as the second sentence. The assumption is that the random sentence will be disconnected from the first sentence.<br>
To help the model distinguish between the two sentences in training, the input is processed in the following way before entering the model:<br>
1. A [CLS] token is inserted at the beginning of the first sentence and a [SEP] token is inserted at the end of each sentence.
2. A sentence embedding indicating Sentence A or Sentence B is added to each token. Sentence embeddings are similar in concept to token embeddings with a vocabulary of 2.
3. A positional embedding is added to each token to indicate its position in the sequence. The concept and implementation of positional embedding are presented in the Transformer paper.
![](https://miro.medium.com/max/1321/0*m_kXt3uqZH9e7H4w.png)<br>

To predict if the second sentence is indeed connected to the first, the following steps are performed:
1. The entire input sequence goes through the Transformer model.
2. The output of the [CLS] token is transformed into a 2×1 shaped vector, using a simple classification layer (learned matrices of weights and biases).
3. Calculating the probability of IsNextSequence with softmax.<br>

When training the BERT model, Masked LM and Next Sentence Prediction are trained together, with the goal of minimizing the combined loss function of the two strategies.

## 3.3 How to use BERT (Fine-tuning):
Using BERT for a specific task is relatively straightforward:<br>
BERT can be used for a wide variety of language tasks, while only adding a small layer to the core model:<br>
1. Classification tasks such as sentiment analysis are done similarly to Next Sentence classification, by adding a classification layer on top of the Transformer output for the [CLS] token.<br>
2. In Question Answering tasks (e.g. SQuAD v1.1), the software receives a question regarding a text sequence and is required to mark the answer in the sequence. Using BERT, a Q&A model can be trained by learning two extra vectors that mark the beginning and the end of the answer.<br>
3. In Named Entity Recognition (NER), the software receives a text sequence and is required to mark the various types of entities (Person, Organization, Date, etc) that appear in the text. Using BERT, a NER model can be trained by feeding the output vector of each token into a classification layer that predicts the NER label.<br>

## 3.4 Takeaways:
1. Model size matters, even at huge scale. BERT_large, with 345 million parameters, is the largest model of its kind. It is demonstrably superior on small-scale tasks to BERT_base, which uses the same architecture with “only” 110 million parameters.<br>
2. With enough training data, more training steps == higher accuracy. For instance, on the MNLI task, the BERT_base accuracy improves by 1.0% when trained on 1M steps (128,000 words batch size) compared to 500K steps with the same batch size.<br>
3. BERT’s bidirectional approach (MLM) converges slower than left-to-right approaches (because only 15% of words are predicted in each batch) but bidirectional training still outperforms left-to-right training after a small number of pre-training steps.<br>

![](https://miro.medium.com/max/1773/0*KONsqvDohE7ytu_E.png)

## 3.5 Compute considerations (training and applying):<br>
![](https://miro.medium.com/max/875/1*LgbpLsRUGbtTPmMSUO2Drw.png)<br>

# 4. Implementation of BERT using TFHub: <br>
#### Credits : This part was taken from [this kernel](https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub) please refer to this awesome kernel and consider upvoting.Thanks to the author of the kernel [xhlulu](https://www.kaggle.com/xhlulu)

## 4.1 Importing necessary modules

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub


/kaggle/working/__notebook__.ipynb


### Getting tokenizer

In [2]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [3]:
import tokenization

## 4.2 Helper Functions:

In [4]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [5]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## 4.3 Loading BERT from the Tensorflow Hub:

In [6]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 1min 22s, sys: 8.49 s, total: 1min 31s
Wall time: 1min 34s


## 4.4 Loading data:

In [7]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

## 4.5 Loading tokenizer from the bert layer:

In [8]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

## 4.6 Encoding the text into tokens, masks, and segment flags:<br>

In [9]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

## 4.7 Model: Build, Train, Predict, Submit:

In [10]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [11]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=16
)

model.save('model.h5')

Train on 6090 samples, validate on 1523 samples
Epoch 1/3
6090/6090 [==============================] - 396s 65ms/sample - loss: 0.4925 - accuracy: 0.7726 - val_loss: 0.4094 - val_accuracy: 0.8280
Epoch 2/3
6090/6090 [==============================] - 354s 58ms/sample - loss: 0.3634 - accuracy: 0.8486 - val_loss: 0.3892 - val_accuracy: 0.8267
Epoch 3/3
6090/6090 [==============================] - 354s 58ms/sample - loss: 0.2948 - accuracy: 0.8783 - val_loss: 0.4241 - val_accuracy: 0.8234


In [12]:
test_pred = model.predict(test_input)

In [13]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)

In [14]:
submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1



# <font color='red'>Please consider Upvoting if you like this kernel.</font> <br>
# Thank you for reading this kernel.<br>
# Happy learning & sharing 😊<br>

<a href="#top" class="btn btn-primary btn-lg active" role="button" aria-pressed="true">Go to TOP</a>